In [ ]:
#import das bibliotecas

import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
import pandas as pd
import re

In [ ]:
nltk.download('stopwords')
nltk.download('rslp')

In [ ]:
import json

# Carregar o JSON
with open("colecao - trabalho 01.json", 'r', encoding='utf-8') as f:
    colecao_documentos = json.load(f)

# Criar DataFrame diretamente a partir da lista de dicionários
df = pd.DataFrame(colecao_documentos)

# Exibir informações sobre o DataFrame
print("Forma do DataFrame:", df.shape)
print("\nPrimeiras linhas:")
print(df.head())
print("\nColunas:", df.columns.tolist())
print("\nTipos de dados:")
print(df.dtypes)
print("\nPrimeiro documento (completo):")
print(df.iloc[0])

In [ ]:
#entender a estrutura do json e padronizar em uma estrutura manipulavel (dicionário ou lista?)


#with open('colecao - trabalho 01.json', 'r', encoding='utf-8') as f:
#    colecao_documentos = json.load(f)
#print(colecao_documentos)

In [ ]:
# Método 1: Simples e direto - usando pd.DataFrame()
# Perfeito quando o JSON é um array de objetos com mesma estrutura

df_simple = pd.DataFrame(colecao_documentos)

print("=== MÉTODO 1: DataFrame Simples ===")
print(df_simple.shape)
print(df_simple.head(3))

In [ ]:
# Método 3: Preparar dados para processamento de NLP

# Criar cópia do DataFrame para pré-processamento
df_processado = df_simple.copy()

# 1. Converter todos os textos para minúsculas
df_processado['content_lower'] = df_processado['content'].str.lower()

# 2. Remover quebras de linha múltiplas
df_processado['content_clean'] = df_processado['content_lower'].str.replace(r'\n+', ' ', regex=True)

# 3. Remover espaços múltiplos
df_processado['content_clean'] = df_processado['content_clean'].str.replace(r'\s+', ' ', regex=True)

# 4. Remover espaços no início e fim
df_processado['content_clean'] = df_processado['content_clean'].str.strip()

print("=== MÉTODO 3: Normalização com Limpeza ===")
print("Forma:", df_processado.shape)
print("\nColunas:", df_processado.columns.tolist())
print("\nPrimeiro documento original:")
print(df_processado['content'].iloc[0][:200])
print("\nPrimeiro documento processado:")
print(df_processado['content_clean'].iloc[0][:200])

In [ ]:
#pré processamento dos dados

# Para usar o sistema completo, execute no terminal:
# python main.py

# Testes rápidos do sistema no notebook:
print("=" * 60)
print("TESTES RÁPIDOS DO SISTEMA DE INDEXAÇÃO")
print("=" * 60)

In [10]:
# Teste 1: Inicializar o gerenciador e adicionar alguns documentos
from gerenciador import GerenciadorColecao
from search_engine import MotorBusca

# Cria gerenciador
gerenciador = GerenciadorColecao()

# Carrega documentos do JSON
documentos = gerenciador.carregar_json("colecao - trabalho 01.json")

# Adiciona os primeiros 5 documentos
print("\n1. ADICIONANDO DOCUMENTOS")
print("-" * 60)
for i in range(min(5, len(documentos))):
    doc = documentos[i]
    gerenciador.adicionar_documento(
        doc_id=i,
        nome=doc["name"],
        conteudo=doc["content"]
    )
    print(f"✓ {doc['name']} adicionado ({len(doc['content'])} caracteres)")

print(f"\nTotal de documentos: {len(gerenciador.documentos)}")
print(f"Total de palavras únicas: {len(gerenciador.vocabulario)}")


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\ferre\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!



1. ADICIONANDO DOCUMENTOS
------------------------------------------------------------
✓ D1 adicionado (1120 caracteres)
✓ D2 adicionado (1080 caracteres)
✓ D3 adicionado (1063 caracteres)
✓ D4 adicionado (958 caracteres)
✓ D5 adicionado (918 caracteres)

Total de documentos: 5
Total de palavras únicas: 222


In [11]:
# Teste 2: Exibir vocabulário e matriz TF-IDF
print("\n2. VOCABULÁRIO")
print("-" * 60)
vocab = gerenciador.obter_vocabulario_ordenado()
print(f"Total de palavras únicas: {len(vocab)}")
print(f"Primeiras 20 palavras: {vocab[:20]}")

print("\n3. MATRIZ TF-IDF (amostra)")
print("-" * 60)
dados, vocab_matriz = gerenciador.obter_matriz_tfidf_tabular()

# Exibe matriz com os primeiros 8 termos
print(f"{'Documento':<12}", end="")
for palavra in vocab_matriz[:8]:
    print(f"{palavra:>12}", end="")
print()
print("-" * (12 + 12 * 8))

for doc_id in sorted(dados.keys()):
    print(f"D{doc_id:<10}", end="")
    for palavra in vocab_matriz[:8]:
        valor = dados[doc_id][palavra]
        if valor > 0:
            print(f"{valor:>12.4f}", end="")
        else:
            print(f"{'-':>12}", end="")
    print()



2. VOCABULÁRIO
------------------------------------------------------------
Total de palavras únicas: 222
Primeiras 20 palavras: ["'estrut", "'fil", "'list", "'tabel", "'árvor", 'abord', 'aceler', 'acess', 'acompanh', 'adapt', 'adequ', 'adic', 'ajud', 'algoritm', 'alt', 'ampli', 'apen', 'aplic', 'aproxim', 'atenç']

3. MATRIZ TF-IDF (amostra)
------------------------------------------------------------
Documento        'estrut        'fil       'list      'tabel      'árvor       abord      aceler       acess
------------------------------------------------------------------------------------------------------------
D0                    -           -           -           -           -           -           -           -
D1                    -      0.0071           -           -      0.0071      0.0071           -           -
D2                    -      0.0043           -           -           -           -      0.0116      0.0085
D3                    -           -           -    

In [12]:
# Teste 3: Índice Invertido
print("\n4. ÍNDICE INVERTIDO (amostra)")
print("-" * 60)
indice = gerenciador.obter_indice_invertido_formatado()
print(f"Total de palavras no índice: {len(indice)}\n")

# Mostra primeiras 10 palavras do índice
for i, palavra in enumerate(list(indice.keys())[:10], 1):
    docs_info = indice[palavra]
    print(f"{i:2}. {palavra:>15} → ", end="")
    for doc_id in sorted(docs_info.keys()):
        posicoes = docs_info[doc_id]
        print(f"D{doc_id}:{posicoes}", end=" | ")
    print()



4. ÍNDICE INVERTIDO (amostra)
------------------------------------------------------------
Total de palavras no índice: 222

 1.         'estrut → D0:[12] | D1:[84] | D2:[86] | 
 2.            'fil → D1:[90] | D2:[89] | 
 3.           'list → D0:[15] | 
 4.          'tabel → D0:[54] | 
 5.          'árvor → D1:[87] | 
 6.           abord → D1:[66] | D4:[52] | 
 7.          aceler → D2:[60] | 
 8.           acess → D0:[32, 62] | D2:[19, 61] | D3:[20, 32] | D4:[55] | 
 9.        acompanh → D4:[21] | 
10.           adapt → D2:[30] | D3:[10] | D4:[29] | 


In [13]:
# Teste 4: Motor de Busca - Busca Booleana
motor_busca = MotorBusca(gerenciador)

print("\n5. BUSCA BOOLEANA")
print("-" * 60)
consulta_booleana = "estrutura AND dados"
resultados = motor_busca.busca_booleana(consulta_booleana)
print(f"Consulta: '{consulta_booleana}'")
print(f"Resultados: {len(resultados)} documento(s)")
for doc_id, nome in resultados:
    print(f"  - {nome}")



5. BUSCA BOOLEANA
------------------------------------------------------------
Consulta: 'estrutura AND dados'
Resultados: 5 documento(s)
  - D1
  - D2
  - D3
  - D4
  - D5


In [17]:
# Teste 5: Busca por Similaridade
print("\n6. BUSCA POR SIMILARIDADE (COSSENO)")
print("-" * 60)
consulta_similaridade = "estrutura de dados"
resultados_sim = motor_busca.busca_similaridade_cosseno(consulta_similaridade, top_k=5)
print(f"Consulta: '{consulta_similaridade}'")
print(f"Top 5 resultados por similaridade:\n")
print(f"{'Rank':<5} {'Doc':<8} {'Similaridade':<15} {'Conteúdo':<30}")
print("-" * 60)
for i, (doc_id, nome, score) in enumerate(resultados_sim, 1):
    conteudo = gerenciador.documentos[doc_id]['content'][:25]
    print(f"{i:<5} {nome:<8} {score:<15.4f} {conteudo}...")



6. BUSCA POR SIMILARIDADE (COSSENO)
------------------------------------------------------------
Consulta: 'estrutura de dados'
Top 5 resultados por similaridade:

Rank  Doc      Similaridade    Conteúdo                      
------------------------------------------------------------


In [15]:
# Teste 6: Busca por Frases
print("\n7. BUSCA POR FRASES")
print("-" * 60)
consulta_frase = "lista encadeada simples"
resultados_frase = motor_busca.busca_por_frases(consulta_frase, top_k=5)
print(f"Consulta: '{consulta_frase}'")
print(f"Resultados encontrados:\n")

if resultados_frase:
    print(f"{'Rank':<5} {'Doc':<8} {'Ocorrências':<15} {'Conteúdo':<30}")
    print("-" * 60)
    for i, (doc_id, nome, score) in enumerate(resultados_frase, 1):
        conteudo = gerenciador.documentos[doc_id]['content'][:25]
        print(f"{i:<5} {nome:<8} {score:<15d} {conteudo}...")
else:
    print("Nenhum resultado encontrado. Tente outro termo.")

# Resumo final
print("\n" + "=" * 60)
print("RESUMO DO TESTE")
print("=" * 60)
stats = gerenciador.obter_estatisticas()
print(f"Total de documentos:        {stats['total_documentos']}")
print(f"Total de palavras únicas:   {stats['total_palavras_unicas']}")
print(f"Total de palavras:          {stats['total_palavras']}")
print(f"Média de palavras por doc:  {stats['media_palavras_por_doc']:.2f}")



7. BUSCA POR FRASES
------------------------------------------------------------
Consulta: 'lista encadeada simples'
Resultados encontrados:

Rank  Doc      Ocorrências     Conteúdo                      
------------------------------------------------------------
1     D2       1               O estudo de estruturas de...
2     D3       1               Projetar algoritmos que l...
3     D4       1               Em muitos sistemas modern...
4     D5       1               Ao projetar uma aplicação...

RESUMO DO TESTE
Total de documentos:        5
Total de palavras únicas:   222
Total de palavras:          463
Média de palavras por doc:  92.60
